# Silver Layer: CRM Customers Transformation
This notebook processes raw customer data from the CRM system (`crm_cust_info`).
- **Framework**: Utilizes `silver_engine` for standardized trimming and ingestion timestamps.
- **Transformations**: 
    - Normalizes Marital Status (S/M -> Single/Married).
    - Normalizes Gender (F/M -> Female/Male).
    - Standardizes `customer_number` to upper case for integration.
- **Output**: Cleaned Delta table `workspace.silver.crm_customers`.


In [0]:
%run ../../helpers/silver_engine.ipynb

In [0]:
%python
import pyspark.sql.functions as F

def logic(df):
    return (
        df
        # 1. Data Quality: Remove records without a valid ID
        .filter(F.col("cst_id").isNotNull())
        
        # 2. Normalization: Marital Status
        .withColumn("marital_status", 
            F.when(F.upper(F.col("cst_marital_status")) == "S", "Single")
             .when(F.upper(F.col("cst_marital_status")) == "M", "Married")
             .otherwise("n/a")
        )
        
        # 3. Normalization: Gender
        .withColumn("gender", 
            F.when(F.upper(F.col("cst_gndr")) == "F", "Female")
             .when(F.upper(F.col("cst_gndr")) == "M", "Male")
             .otherwise("n/a")
        )
        
        # 4. Standardization: Key formatting and column renaming
        .withColumn("customer_number", F.upper(F.col("cst_key")))
        .withColumnRenamed("cst_id", "customer_id")
        .withColumnRenamed("cst_firstname", "first_name")
        .withColumnRenamed("cst_lastname", "last_name")
        .withColumnRenamed("cst_create_date", "created_date")
        
        # 5. Final Selection: Selecting only the necessary columns for Silver layer
        .select("customer_id", "customer_number", "first_name", "last_name", "marital_status", "gender", "created_date")
    )

# Executing the automated silver pipeline
run_silver_pipeline("crm_cust_info", "crm_customers", logic)

In [0]:
%sql
-- Final validation to ensure data quality and schema consistency
SELECT * FROM workspace.silver.crm_customers LIMIT 10;